In [80]:
# -*- coding: utf-8 -*-
%matplotlib inline

import pandas as pd
import numpy as np
import math
import matplotlib as mplstyle
import matplotlib.pyplot as plt
import locale
import arrow
from locale import atof
from dateutil import parser
from datetime import datetime

### Imports

In [81]:
df_vacc = pd.read_csv('../data/imports/dtp1.csv')
df_pop = pd.read_csv('../data/imports/population-under14.csv')
df_gdp = pd.read_csv('../data/imports/GDP2016.csv')
df_ppp = pd.read_csv('../data/imports/PPP.csv')
df_pppMeta = pd.read_csv('../data/imports/PPP_meta.csv')

### Analysis

In [82]:
# Merge vaccine data and GDP
df_vacGdpMerge = df_vacc.merge(df_gdp, on='iso3', how='outer')

In [83]:
# Prep population for merge 
df_pop = df_pop.rename(columns = {'Region, country or area' : 'country'})

# Merge vacGdp and population
df_vacGdpPopMerge = df_vacGdpMerge.merge(df_pop, on='country', how='outer')

In [84]:
# Clean merge
df_vacGdpPopMergeClean = df_vacGdpPopMerge.drop(['vaccine', 'economy', ' Male ', ' Female '], axis=1)
df_vacGdpPopMergeClean = df_vacGdpPopMergeClean.rename(columns = {'millions of US dollars':'GDP',' 0-14 (percentage) ':'Pop % under 14'})
# df_vacGdpPopMergeClean

In [85]:
# Columns for condesnced df
cols = ['unicef_region','iso3','country','GDP','2016','1996','1986','Pop % under 14']

# Apply columns
df_vacGdpPopMergeCond = df_vacGdpPopMergeClean[cols]
df_vacGdpPopMergeCond = df_vacGdpPopMergeCond[np.isfinite(df_vacGdpPopMergeCond['2016'])]
df_vacGdpPopMergeCond.sort_values('GDP')
# df_vacGdpPopMergeCond.to_csv('../data/exports/df_vacGdpPopMergeCond.csv')

,unicef_region,iso3,country,GDP,2016,1996,1986,Pop % under 14
97,Middle East & North Africa,LBY,Libya,..,98.0,98.0,94.0,28.0
111,Western Europe,MCO,Monaco,..,99.0,99.0,NaN,NaN
147,Western Europe,SMR,San Marino,..,78.0,99.0,99.0,NaN
46,East Asia & Pacific,PRK,Democratic People�s Republic of Korea,..,97.0,74.0,82.0,NaN
169,Middle East & North Africa,SYR,Syrian Arab Republic,..,61.0,96.0,90.0,37.0
56,Eastern & Southern Africa,ERI,Eritrea,..,97.0,75.0,NaN,42.0
3,Western Europe,AND,Andorra,..,99.0,NaN,NaN,NaN
190,Latin America & Caribbean,VEN,Venezuela (Bolivarian Republic of),..,98.0,80.0,80.0,28.0
68,Latin America & Caribbean,GRD,Grenada,"1,016",98.0,93.0,98.0,26.0
109,Latin America & Caribbean,MEX,Mexico,"1,045,998",99.0,98.0,75.0,27.0


In [86]:
# Prep to merge PPP and PPP Meta

# Raname columns for merge
df_ppp = df_ppp.rename(columns = {'Country Code':'iso3','2016':'PPP 2016'})
df_pppMeta = df_pppMeta.rename(columns = {'Country Code':'iso3'})

# Define columns to keep
pppCols = ['iso3','PPP 2016']
pppMetaCols = ['iso3','IncomeGroup']

# Apply columns to dataframes
df_pppCond = df_ppp[pppCols]
df_pppMetaCond = df_pppMeta[pppMetaCols]

# Merge PPP files together
df_pppMerge = df_pppCond.merge(df_pppMetaCond,on='iso3',how='outer')

In [87]:
df_pppMerge

,iso3,PPP 2016,IncomeGroup
0,ABW,NaN,High income
1,AFG,1876.544682,Low income
2,AGO,6499.067104,Lower middle income
3,ALB,11928.536560,Upper middle income
4,AND,NaN,High income
5,ARB,16697.596840,NaN
6,ARE,72418.610860,High income
7,ARG,19934.372490,Upper middle income
8,ARM,8817.953996,Lower middle income
9,ASM,NaN,Upper middle income


In [88]:
# Merge df_vacGdpPopMergeCond and df_pppMerge then rename columns
df_finalMerge = df_vacGdpPopMergeCond.merge(df_pppMerge, on='iso3', how='outer')
df_finalMergeClean = df_finalMerge.rename(columns = {'GDP':'GDP 2016','2016':'DTP1-16','1996':'DTP1-96','1986':'DTP1-86'})
# list(df_finalMerge)

# Filter for blanks
df_finalMergeClean = df_finalMergeClean[np.isfinite(df_finalMerge['2016'])]

# Reorder
list(df_finalMergeClean)
cols = ['unicef_region','iso3','country','IncomeGroup','GDP 2016','PPP 2016','Pop % under 14','DTP1-16','DTP1-96','DTP1-86']
df_finalMergeClean = df_finalMergeClean[cols]

df_finalMergeClean.to_csv('../data/exports/df_finalMergeClean.csv')

In [90]:
df_finalMergeClean.sort_values('country')

,unicef_region,iso3,country,IncomeGroup,GDP 2016,PPP 2016,Pop % under 14,DTP1-16,DTP1-96,DTP1-86
0,South Asia,AFG,Afghanistan,Low income,"19,469",1876.544682,43.0,73.0,56.0,32.0
1,Eastern Europe & Central Asia,ALB,Albania,Upper middle income,"11,927",11928.536560,17.0,99.0,98.0,98.0
2,Middle East & North Africa,DZA,Algeria,Upper middle income,"156,080",15074.924810,29.0,96.0,96.0,92.0
3,Western Europe,AND,Andorra,High income,..,NaN,NaN,99.0,NaN,NaN
4,Eastern & Southern Africa,AGO,Angola,Lower middle income,"89,633",6499.067104,47.0,79.0,53.0,31.0
5,Latin America & Caribbean,ATG,Antigua and Barbuda,High income,"1,449",22413.483800,24.0,99.0,99.0,98.0
6,Latin America & Caribbean,ARG,Argentina,Upper middle income,"545,866",19934.372490,25.0,97.0,94.0,86.0
7,Eastern Europe & Central Asia,ARM,Armenia,Lower middle income,"10,547",8817.953996,20.0,97.0,95.0,NaN
8,East Asia & Pacific,AUS,Australia,High income,"1,204,616",46789.927240,19.0,98.0,95.0,91.0
9,Western Europe,AUT,Austria,High income,"386,428",50077.832610,14.0,99.0,96.0,96.0
